In [5]:
import os
import pandas as pd
import numpy as np
import math

In [6]:
def colunm_change(str_):
    if 'YR' in str_:
        return str_[2:]
    else:
        return str_

def table_name(file_name):
    aux = file_name.split("_")[-1]
    str_ = aux.split('.')[:-1]
    return "_".join(str_)

In [7]:
# files = {'SL.EMP.VULN.FE.ZS.csv',
# 'SL.EMP.VULN.MA.ZS.csv', 'SL.TLF.ACTI.1524.FE.ZS.csv', 'SL.TLF.ACTI.1524.MA.ZS.csv',
# 'SL.TLF.ACTI.1524.ZS.csv', 'SL.TLF.ACTI.FE.ZS.csv', 'SL.TLF.ACTI.MA.ZS.csv',
# 'SL.TLF.ACTI.ZS.csv', 'SL.TLF.CACT.FE.ZS.csv', 'SL.TLF.CACT.MA.ZS.csv',
# 'SL.UEM.TOTL.FE.ZS.csv', 'SL.UEM.TOTL.MA.ZS.csv', 'SP.ADO.TFRT.csv',
# 'SP.DYN.LE00.FE.IN.csv', 'SP.DYN.LE00.MA.IN.csv'}

files = {'SL.EMP.VULN.FE.ZS.csv',
'SL.TLF.ACTI.1524.MA.ZS.csv',
'SL.TLF.ACTI.ZS.csv', 'SL.TLF.CACT.FE.ZS.csv', 'SL.TLF.CACT.MA.ZS.csv',
'SL.UEM.TOTL.FE.ZS.csv', 'SL.UEM.TOTL.MA.ZS.csv', 'SP.ADO.TFRT.csv',
'SP.DYN.LE00.FE.IN.csv', 'SP.DYN.LE00.MA.IN.csv'}

out_df = pd.DataFrame()
for file in files:
    df_aux = pd.read_csv('data/'+file)
    df_aux.columns = [colunm_change(c) for c in df_aux.columns]

    out = pd.melt(df_aux, id_vars=['economy',],
                var_name='Year', value_name=table_name(file))
    

    if(len(out_df) == 0):
        out_df = out
    else:
        out_df = out_df.merge(out)

out_df.loc[:, 'Year'] = out_df.loc[:, 'Year'].astype(int)

In [8]:
idh_df = pd.read_csv('human-development-index.csv')
idh_df.rename(columns={'Code':'economy', 'Human Development Index' : 'HDI'}, inplace=True)

In [9]:
merge_df = idh_df.merge(out_df)
merge_df

,Entity,economy,Year,HDI,SL_UEM_TOTL_MA_ZS,SP_ADO_TFRT,SL_UEM_TOTL_FE_ZS,SL_TLF_ACTI_ZS,SP_DYN_LE00_MA_IN,SL_TLF_ACTI_1524_MA_ZS,SL_TLF_CACT_FE_ZS,SP_DYN_LE00_FE_IN,SL_EMP_VULN_FE_ZS,SL_TLF_CACT_MA_ZS
0,Afghanistan,AFG,1990,0.284,NaN,142.960,NaN,NaN,43.709,NaN,NaN,48.397,NaN,NaN
1,Afghanistan,AFG,1991,0.292,7.435,147.525,10.433,48.039,44.353,62.904,15.635,49.144,99.237026,79.457
2,Afghanistan,AFG,1992,0.299,7.430,147.521,10.427,48.031,45.070,62.746,15.698,50.320,99.185699,79.402
3,Afghanistan,AFG,1993,0.307,7.435,147.896,10.432,48.000,50.216,62.524,15.715,52.739,99.096507,79.298
4,Afghanistan,AFG,1994,0.300,7.435,155.669,10.432,47.957,49.531,62.237,15.726,53.544,99.048388,79.154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,Zimbabwe,ZWE,2018,0.564,6.984,98.507,6.555,67.052,58.968,57.975,60.683,63.594,78.003529,72.173
5804,Zimbabwe,ZWE,2019,0.560,7.760,97.354,6.990,66.938,58.644,57.301,60.730,63.661,76.308652,71.927
5805,Zimbabwe,ZWE,2020,0.554,8.809,96.180,8.494,66.259,58.129,55.713,59.848,63.862,73.010199,71.431
5806,Zimbabwe,ZWE,2021,0.549,9.223,94.312,9.853,66.562,56.232,54.021,60.147,62.045,69.010063,71.753


In [10]:
merge_df.loc[:, 'desenv'] = merge_df.loc[:, 'HDI'].apply(lambda x: 0 if x < 0.5 else 1) #transforma em 0 - subdesenvolvido e 1 - desenvolvido
merge_df.dropna(subset='HDI', inplace=True) #remove as linhas nulas a partir de HDI nossa label
merge_df.reset_index(inplace=True, drop=True)

merge_df.isna().sum()

Entity                      0
economy                     0
Year                        0
HDI                         0
SL_UEM_TOTL_MA_ZS         453
SP_ADO_TFRT               193
SL_UEM_TOTL_FE_ZS         453
SL_TLF_ACTI_ZS            453
SP_DYN_LE00_MA_IN         267
SL_TLF_ACTI_1524_MA_ZS    453
SL_TLF_CACT_FE_ZS         453
SP_DYN_LE00_FE_IN         267
SL_EMP_VULN_FE_ZS         453
SL_TLF_CACT_MA_ZS         453
desenv                      0
dtype: int64

In [11]:
# remover colunas com muitos dados faltantes
nans = pd.Series(merge_df.isna().sum())

for i in nans.index:
    if nans[i] > 500:
        merge_df.drop(columns = [i], inplace = True)

merge_df.isna().sum()    

Entity                      0
economy                     0
Year                        0
HDI                         0
SL_UEM_TOTL_MA_ZS         453
SP_ADO_TFRT               193
SL_UEM_TOTL_FE_ZS         453
SL_TLF_ACTI_ZS            453
SP_DYN_LE00_MA_IN         267
SL_TLF_ACTI_1524_MA_ZS    453
SL_TLF_CACT_FE_ZS         453
SP_DYN_LE00_FE_IN         267
SL_EMP_VULN_FE_ZS         453
SL_TLF_CACT_MA_ZS         453
desenv                      0
dtype: int64

In [12]:
# out_df.dropna(subset='updown', inplace=True) #remove as linhas nulas de updwon(valor mais recente) nossa label
merge_df.dropna(inplace=True)
merge_df.reset_index(inplace=True, drop=True)

HDI_results = merge_df.drop(columns = ['Entity', 'economy', 'Year', 'desenv'])
class_results = merge_df.drop(columns = ['Entity', 'economy', 'Year', 'HDI'])

HDI_results.to_csv('HDI_results.csv', index=False)
class_results.to_csv('class_results.csv', index=False)


In [13]:
HDI_results

,HDI,SL_UEM_TOTL_MA_ZS,SP_ADO_TFRT,SL_UEM_TOTL_FE_ZS,SL_TLF_ACTI_ZS,SP_DYN_LE00_MA_IN,SL_TLF_ACTI_1524_MA_ZS,SL_TLF_CACT_FE_ZS,SP_DYN_LE00_FE_IN,SL_EMP_VULN_FE_ZS,SL_TLF_CACT_MA_ZS
0,0.292,7.435,147.525,10.433,48.039,44.353,62.904,15.635,49.144,99.237026,79.457
1,0.299,7.430,147.521,10.427,48.031,45.070,62.746,15.698,50.320,99.185699,79.402
2,0.307,7.435,147.896,10.432,48.000,50.216,62.524,15.715,52.739,99.096507,79.298
3,0.300,7.435,155.669,10.432,47.957,49.531,62.237,15.726,53.544,99.048388,79.154
4,0.318,7.428,156.770,10.423,47.920,51.039,61.888,15.796,54.087,98.988570,78.961
...,...,...,...,...,...,...,...,...,...,...,...
5172,0.552,6.410,98.265,6.289,67.093,58.208,58.392,60.646,62.956,79.187622,72.373
5173,0.564,6.984,98.507,6.555,67.052,58.968,57.975,60.683,63.594,78.003529,72.173
5174,0.560,7.760,97.354,6.990,66.938,58.644,57.301,60.730,63.661,76.308652,71.927
5175,0.554,8.809,96.180,8.494,66.259,58.129,55.713,59.848,63.862,73.010199,71.431


In [14]:
class_results

,SL_UEM_TOTL_MA_ZS,SP_ADO_TFRT,SL_UEM_TOTL_FE_ZS,SL_TLF_ACTI_ZS,SP_DYN_LE00_MA_IN,SL_TLF_ACTI_1524_MA_ZS,SL_TLF_CACT_FE_ZS,SP_DYN_LE00_FE_IN,SL_EMP_VULN_FE_ZS,SL_TLF_CACT_MA_ZS,desenv
0,7.435,147.525,10.433,48.039,44.353,62.904,15.635,49.144,99.237026,79.457,0
1,7.430,147.521,10.427,48.031,45.070,62.746,15.698,50.320,99.185699,79.402,0
2,7.435,147.896,10.432,48.000,50.216,62.524,15.715,52.739,99.096507,79.298,0
3,7.435,155.669,10.432,47.957,49.531,62.237,15.726,53.544,99.048388,79.154,0
4,7.428,156.770,10.423,47.920,51.039,61.888,15.796,54.087,98.988570,78.961,0
...,...,...,...,...,...,...,...,...,...,...,...
5172,6.410,98.265,6.289,67.093,58.208,58.392,60.646,62.956,79.187622,72.373,1
5173,6.984,98.507,6.555,67.052,58.968,57.975,60.683,63.594,78.003529,72.173,1
5174,7.760,97.354,6.990,66.938,58.644,57.301,60.730,63.661,76.308652,71.927,1
5175,8.809,96.180,8.494,66.259,58.129,55.713,59.848,63.862,73.010199,71.431,1
